In [1]:
import mip
import numpy as np

In [25]:
schedules = [
    [ # Hi
        [0] * 5,
        [1] * 5,
        [0] * 5,
        [0] * 5,
        [0] * 5,
    ],
    [ # Ta
        [0] * 5,
        [0] * 5,
        [1] * 5,
        [0] * 5,
        [1] * 5,
    ],
    [ # Ni
        [0] * 5,
        [0] * 5,
        [0,1,1,1,1],
        [0] * 5,
        [0] * 5,
    ],
    [ # Um
        [0,0,1,1,1],
        [1,1,1,1,0],
        [1] * 5,
        [0] * 5,
        [1,1,1,1,0]
    ],
    [ # Te
        [0] * 5,
        [1,1,0,0,0],
        [0] * 5,
        [0,0,0,1,1],
        [1,1,0,0,0]
    ],
    [ # Os
        [1] * 5,
        [1,1,0,0,0],
        [0] * 5,
        [0,0,0,1,1],
        [1] * 5
    ],
    [# Su
        [1,1,0,0,0],
        [0,1,1,1,1],
        [1] * 5,
        [1] * 5,
        [1] * 5,
    ],
    [ # Ko
        [0] * 5,
        [0] * 5,
        [0] * 5,
        [0,0,0,1,1],
        [0,0,0,1,1]
    ],
    [ # Na
        [0] * 5,
        [0] * 5,
        [1] * 5,
        [0,0,0,1,1],
        [0,0,0,1,1]
    ]
]

In [26]:
print(len(schedules))
print(len(schedules[0]))
print(len(schedules[0][0]))

9
5
5


In [55]:
m = mip.Model()

n_people = len(schedules)
n_days = len(schedules[0])
n_shifts = len(schedules[0][0])

# Variables
shifts = []
for d_i in range(n_days):
    shifts.append([])
    for s_i in range(n_shifts):
        shifts[d_i].append([])
        for p_i in range(n_people):
            shifts[d_i][s_i].append(m.add_var(var_type=mip.BINARY))

# Constraints
# 各人のスケジュールに合わせる。
for p_i in range(n_people):
    for d_i in range(n_days):
        for s_i in range(n_shifts):
            if (schedules[p_i][d_i][s_i] == 0):
                # シフトに入れない
                m += shifts[d_i][s_i][p_i] == 0

# シフトは埋めないといけない
# かつ、同じシフトに複数人入れない
for d_i in range(n_days):
    for s_i in range(n_shifts):
        m += mip.xsum(shifts[d_i][s_i]) == 1

# 一人当たりのシフト数は最大5つ
for p_i in range(n_people):
    m += mip.xsum(mip.xsum(shifts[d_i][s_i][p_i] for s_i in range(n_shifts)) for d_i in range(n_days)) <= 5

# Objective
# 隣同士のシフトに入る人は、なるべく同じになるようにする
rewards = 0
for p_i in range(n_people):
    for d_i in range(n_days):
        rewards += mip.xsum((shifts[d_i][s_i][p_i] == 1) and (shifts[d_i][s_i+1][p_i] == 1) for s_i in range(n_shifts-1))

# 一つだけの枠にはペナルティ
penalty = 0
for d_i in range(n_days):
    for p_i in range(n_people):
        penalty += mip.xsum(((shifts[d_i][s_i][p_i] == 1) and (shifts[d_i][s_i+1][p_i] == 0)) or ((shifts[d_i][s_i][p_i] == 0) and (shifts[d_i][s_i+1][p_i] == 1)) for s_i in range(n_shifts-1))

m.objective = mip.maximize(rewards - penalty)

In [56]:
m.optimize()

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 22 (-148) rows, 64 (-161) columns and 128 (-458) elements
Clp1000I sum of infeasibilities 8.88178e-16 - average 4.03717e-17, 32 fixed columns
Coin0506I Presolve 15 (-7) rows, 28 (-36) columns and 56 (-72) elements
Clp0006I 0  Obj -0
Clp0029I End of values pass after 28 iterations
Clp0000I Optimal - objective value 0
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0006I 0  Obj -0
Clp0000I Optimal - objective value 0
Clp0000I Optimal - objective value 0
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0 - 0 iterations time 0.002, Presolve 0.00

Starting MIP optimization
Cgl0002I 136 variables fixed
Cgl0004I processed model has 22 rows, 76 columns (76 integer (74 of which binary)) and 140 elements
Coin3009W Conflict 

<OptimizationStatus.OPTIMAL: 0>

In [57]:
for d_i in range(n_days):
    print("Day:", d_i)
    for s_i in range(n_shifts):
        print("Shift:", s_i, end=" assigned to ")
        for p_i in range(n_people):
            if shifts[d_i][s_i][p_i].x == 1:
                print(p_i)

Day: 0
Shift: 0 assigned to 6
Shift: 1 assigned to 6
Shift: 2 assigned to 5
Shift: 3 assigned to 5
Shift: 4 assigned to 5
Day: 1
Shift: 0 assigned to 0
Shift: 1 assigned to 0
Shift: 2 assigned to 0
Shift: 3 assigned to 0
Shift: 4 assigned to 0
Day: 2
Shift: 0 assigned to 8
Shift: 1 assigned to 2
Shift: 2 assigned to 2
Shift: 3 assigned to 2
Shift: 4 assigned to 2
Day: 3
Shift: 0 assigned to 6
Shift: 1 assigned to 6
Shift: 2 assigned to 6
Shift: 3 assigned to 7
Shift: 4 assigned to 7
Day: 4
Shift: 0 assigned to 1
Shift: 1 assigned to 4
Shift: 2 assigned to 1
Shift: 3 assigned to 7
Shift: 4 assigned to 7


In [58]:
for p_i in range(n_people):
    print("Person:", p_i)
    for d_i in range(n_days):
        print("Day:", d_i, end=" assigned to ")
        for s_i in range(n_shifts):
            if shifts[d_i][s_i][p_i].x == 1:
                print(s_i, end=" ")
        print("")

Person: 0
Day: 0 assigned to 
Day: 1 assigned to 0 1 2 3 4 
Day: 2 assigned to 
Day: 3 assigned to 
Day: 4 assigned to 
Person: 1
Day: 0 assigned to 
Day: 1 assigned to 
Day: 2 assigned to 
Day: 3 assigned to 
Day: 4 assigned to 0 2 
Person: 2
Day: 0 assigned to 
Day: 1 assigned to 
Day: 2 assigned to 1 2 3 4 
Day: 3 assigned to 
Day: 4 assigned to 
Person: 3
Day: 0 assigned to 
Day: 1 assigned to 
Day: 2 assigned to 
Day: 3 assigned to 
Day: 4 assigned to 
Person: 4
Day: 0 assigned to 
Day: 1 assigned to 
Day: 2 assigned to 
Day: 3 assigned to 
Day: 4 assigned to 1 
Person: 5
Day: 0 assigned to 2 3 4 
Day: 1 assigned to 
Day: 2 assigned to 
Day: 3 assigned to 
Day: 4 assigned to 
Person: 6
Day: 0 assigned to 0 1 
Day: 1 assigned to 
Day: 2 assigned to 
Day: 3 assigned to 0 1 2 
Day: 4 assigned to 
Person: 7
Day: 0 assigned to 
Day: 1 assigned to 
Day: 2 assigned to 
Day: 3 assigned to 3 4 
Day: 4 assigned to 3 4 
Person: 8
Day: 0 assigned to 
Day: 1 assigned to 
Day: 2 assigned to 0 
